### **5.6 - Scale out Apache Airflow with Celery Executors and Redis**

La mayoría de las veces los local executors pueden ser más que suficientes para tus jobs. Pero en algún momento, si tienes como cientos de DAGs que ejecutar o si tienes una cantidad creciente sostenible de DAGs con los que lidiar, puede ser necesario que te muevas al siguiente paso distribuyendo tu carga de trabajo (workload) entre diferentes nodos worker.  Eso es exactamente lo que el Celery Executor te permite hacer.  Entonces, ¿qué es el Celery Executor?: 

<center><img src="https://i.postimg.cc/dVVtH5Bv/a814.png"></center>

Como puedes adivinar por su nombre, este executor está respaldado por Celery. Celery es una cola asíncrona (asynchronous queue) basada en el paso distribuido de mensajes con el fin de distribuir la carga (load) en un pool de workers.  Un worker se crea utilizando el comando "airflow worker" iniciando un Airflow daemon que gestiona de uno a varios procesos Celery con el fin de ejecutar las tareas de un DAG dado. Así, a diferencia del local executor, en el que se creaban varios subprocesos para ejecutar las tareas en una única máquina, ahora las tareas se reparten entre varias máquinas en las que se siguen creando subprocesos para completar estas tareas de forma concurrente. La mayor ventaja de utilizar el Celery Executor es que cada vez que usted está corto de recursos sólo tiene que añadir un nuevo nodo worker a su clúster y al instante obtener más recursos con el fin de ejecutar sus jobs.  Es lo que llamamos escalado horizontal.  Otra gran ventaja de utilizar el Celery Executor es que, si un worker se cae, el Celery worker se adapta rápidamente y es capaz de asignar tareas a otro worker.  Este no es el caso con el Local Executor donde si tu nodo se cae, entonces Airflow se cae también.  Así que proporciona una especie de alta disponibilidad para sus workers.  Si un worker se cae, Airflow puede seguir programando tareas (schedule tasks) en otros workers y, una vez que vuelva a funcionar, empezar a reprogramar tareas (rescheduling tasks) en él.  Una cosa a tener en cuenta es que usted tendrá que añadir un nuevo componente a su arquitectura Airflow que es un sistema de corredor de mensajes (message broker system).  De hecho, con el fin de distribuir las tareas a los workers, Celery necesita un corredor de mensajes como Redis o RabbitMQ donde las tareas serán empujadas a una o varias colas.  A continuación, cada worker extraerá una tarea de una cola determinada y comenzará a ejecutar el código proporcionado.  Permítanme mostrarles un esquema de la arquitectura con la que terminarán.  

<center><img src="https://i.postimg.cc/3wyr7WVc/a815.png"></center>

Así que básicamente tenemos un nodo maestro (master node) donde el scheduler y el web server están ubicados. Cuando el scheduler encuentra que un DAG necesita ser disparado, crea una nueva instancia de DAGRun en la metadata database que puede ser MySQL o PostgreSQL por ejemplo.  A continuación, comienza a disparar las tareas individuales en el DAG empujando mensajes (pushing messages) que contienen información sobre las tareas a ejecutar en el sistema de corredor de mensajes (message broker system). Puede ser RabbitMQ o Redis. Por cierto, no hay una mejor práctica como cuál debe utilizar, pero en mi opinión, si ya ha instalado RabbitMQ entonces vaya por ella.  De lo contrario, puede ser interesante echar un vistazo a Redis, ya que puede ser más fácil de mantener y utilizar. Dado que los workers de Celery extraen periódicamente mensajes del sistema de colas (pull messages from the queuing system), una vez que un mensaje con información de la tarea a ejecutar se introduce en una cola, es extraído por un worker determinado.  Este worker actualiza el estado de esa tarea en la metadata database y la ejecuta.  Finalmente, cuando la tarea termina, el worker actualiza el estado de la tarea y el trabajo está hecho.  Con este tipo de arquitectura puedes dedicar algunos de tus workers a ejecutar tareas específicas en función de sus recursos.  Por ejemplo, digamos que tienes una tarea que es muy intensiva en CPU y tienes un nodo worker con más recursos de CPU, entonces podrías decir, esta tarea sólo debe ser ejecutada en ese worker.  Esto también puede ser muy útil si tienes dependencias que sólo están instaladas en un nodo específico y son requeridas por una tarea determinada.  Así que con lo que acabamos de descubrir, se puede entender por qué el Celery Executor se recomienda en la producción.  Hace que su instancia de Airflow sea escalable, así como altamente disponible, al menos para sus workers y usted obtiene un mejor control sobre cómo se deben ejecutar sus tareas.  Ahora la pregunta es, ¿cuáles son los inconvenientes?  

<center><img src="https://i.postimg.cc/cLk1Vf93/a816.png"></center>

Bueno, como has visto, tienes que instalar un servicio de mensajería (message broker service), lo que significa que tienes que aprender, utilizar y mantener RabbitMQ o Redis. Dado que la arquitectura es más compleja de configurar que con el Local Executor, lleva algo de trabajo y tiempo configurarlo y supervisarlo correctamente.  Además, tener la posibilidad de escalar horizontalmente tu cluster añadiendo nuevos workers significa que tienes que mantenerlos. Como puedes imaginar, si tienes veinte workers diferentes que supervisar y mantener, es mucho más complicado que con un solo nodo. Tendrás que lidiar con dependencias perdidas, workers que se caen, problemas de seguridad y de red, etcétera.  Así que, de nuevo, trabajar en un entorno distribuido puede ser una auténtica pesadilla si tu equipo no está preparado para ello.  Es por eso que recomiendo encarecidamente que primero vayas con el LocalExecutor y luego, si después de monitorizar Airflow ves que necesitas más recursos para hacer frente a tus DAGs, ve con el CeleryExecutor.  Muy bien, ahora usted tiene una mejor comprensión de lo que es el Celery Executor, cómo funciona y por qué podría ser útil, vamos a ver un ejemplo de ello en el siguiente video.  